Build a movie recommendator

In [128]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Import the data

In [129]:
data = pd.read_csv('_src/data/movies_normalized.csv', error_bad_lines = False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11564\1379505799.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('_src/data/movies_normalized.csv', error_bad_lines = False)


show the data

In [130]:
data

,collection_name,directors,genres_list,id,title,prod_companies,prod_countries,original_language,release_language,overview,...,release_date,release_year,runtime,status,tagline,vote_average,vote_count,budget,revenue,return
0,Toy Story Collection,['John Lasseter'],"['Animation', 'Comedy', 'Family']",862,Toy Story,['Pixar Animation Studios'],['United States of America'],en,English,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,1995,81.0,Released,NaN,7.7,5415.0,30000000,373554033,12.45
1,0,['Joe Johnston'],"['Adventure', 'Fantasy', 'Family']",8844,Jumanji,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],en,English,When siblings Judy and Peter discover an encha...,...,1995-12-15,1995,104.0,Released,Roll the dice and unleash the excitement!,6.9,2413.0,65000000,262797249,4.04
2,Grumpy Old Men Collection,['Howard Deutch'],"['Romance', 'Comedy']",15602,Grumpier Old Men,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],en,English,A family wedding reignites the ancient feud be...,...,1995-12-22,1995,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,0,0,NaN
3,0,['Forest Whitaker'],"['Comedy', 'Drama', 'Romance']",31357,Waiting to Exhale,['Twentieth Century Fox Film Corporation'],['United States of America'],en,English,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,1995,127.0,Released,Friends are the people who let you be yourself...,6.1,34.0,16000000,81452156,5.09
4,Father of the Bride Collection,['Charles Shyer'],['Comedy'],11862,Father of the Bride Part II,"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],en,English,Just when George Banks has recovered from his ...,...,1995-02-10,1995,106.0,Released,Just When His World Is Back To Normal... He's ...,5.7,173.0,0,76578911,76578911.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,0,['John Irvin'],"['Drama', 'Action', 'Romance']",30840,Robin Hood,"['Westdeutscher Rundfunk (WDR)', 'Working Titl...","['Canada', 'Germany', 'United Kingdom', 'Unite...",en,English,"Yet another version of the classic epic, with ...",...,1991-05-13,1991,104.0,Released,NaN,5.7,26.0,0,0,NaN
45447,0,['Lav Diaz'],['Drama'],111109,Century of Birthing,['Sine Olivia'],['Philippines'],tl,NaN,An artist struggles to finish his work while a...,...,2011-11-17,2011,360.0,Released,NaN,9.0,3.0,0,0,NaN
45448,0,['Mark L. Lester'],"['Action', 'Drama', 'Thriller']",67758,Betrayal,['American World Pictures'],['United States of America'],en,English,"When one of her hits goes wrong, a professiona...",...,2003-08-01,2003,90.0,Released,A deadly game of wits.,3.8,6.0,0,0,NaN
45449,0,['Yakov Protazanov'],NaN,227506,Satan Triumphant,['Yermoliev'],['Russia'],en,NaN,"In a small town live two brothers, one a minis...",...,1917-10-21,1917,87.0,Released,NaN,0.0,0.0,0,0,NaN


We keep the important columns

In [147]:
#Create a function to combine the important features
def combine_features(feature_str):
    return str(feature_str)

In [132]:
columns = ['title','genres_list','prod_companies','vote_average']

In [148]:
data['combine_features'] = data['directors'].apply(combine_features) + ' '
data['combine_features'] += data['title'].apply(combine_features) + ' '
data['combine_features'] += data['genres_list'].apply(combine_features) + ' '
data['combine_features'] += data['prod_companies'].apply(combine_features) + ' '
data['combine_features'] += data['vote_average'].apply(combine_features) 

In [149]:
data

,collection_name,directors,genres_list,id,title,prod_companies,prod_countries,original_language,release_language,overview,...,release_year,runtime,status,tagline,vote_average,vote_count,budget,revenue,return,combine_features
0,Toy Story Collection,['John Lasseter'],"['Animation', 'Comedy', 'Family']",862,Toy Story,['Pixar Animation Studios'],['United States of America'],en,English,"Led by Woody, Andy's toys live happily in his ...",...,1995,81.0,Released,NaN,7.7,5415.0,30000000,373554033,12.45,"['John Lasseter'] Toy Story ['Animation', 'Com..."
1,0,['Joe Johnston'],"['Adventure', 'Fantasy', 'Family']",8844,Jumanji,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],en,English,When siblings Judy and Peter discover an encha...,...,1995,104.0,Released,Roll the dice and unleash the excitement!,6.9,2413.0,65000000,262797249,4.04,"['Joe Johnston'] Jumanji ['Adventure', 'Fantas..."
2,Grumpy Old Men Collection,['Howard Deutch'],"['Romance', 'Comedy']",15602,Grumpier Old Men,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],en,English,A family wedding reignites the ancient feud be...,...,1995,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,0,0,NaN,"['Howard Deutch'] Grumpier Old Men ['Romance',..."
3,0,['Forest Whitaker'],"['Comedy', 'Drama', 'Romance']",31357,Waiting to Exhale,['Twentieth Century Fox Film Corporation'],['United States of America'],en,English,"Cheated on, mistreated and stepped on, the wom...",...,1995,127.0,Released,Friends are the people who let you be yourself...,6.1,34.0,16000000,81452156,5.09,['Forest Whitaker'] Waiting to Exhale ['Comedy...
4,Father of the Bride Collection,['Charles Shyer'],['Comedy'],11862,Father of the Bride Part II,"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],en,English,Just when George Banks has recovered from his ...,...,1995,106.0,Released,Just When His World Is Back To Normal... He's ...,5.7,173.0,0,76578911,76578911.00,['Charles Shyer'] Father of the Bride Part II ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,0,['John Irvin'],"['Drama', 'Action', 'Romance']",30840,Robin Hood,"['Westdeutscher Rundfunk (WDR)', 'Working Titl...","['Canada', 'Germany', 'United Kingdom', 'Unite...",en,English,"Yet another version of the classic epic, with ...",...,1991,104.0,Released,NaN,5.7,26.0,0,0,NaN,"['John Irvin'] Robin Hood ['Drama', 'Action', ..."
45447,0,['Lav Diaz'],['Drama'],111109,Century of Birthing,['Sine Olivia'],['Philippines'],tl,NaN,An artist struggles to finish his work while a...,...,2011,360.0,Released,NaN,9.0,3.0,0,0,NaN,['Lav Diaz'] Century of Birthing ['Drama'] ['S...
45448,0,['Mark L. Lester'],"['Action', 'Drama', 'Thriller']",67758,Betrayal,['American World Pictures'],['United States of America'],en,English,"When one of her hits goes wrong, a professiona...",...,2003,90.0,Released,A deadly game of wits.,3.8,6.0,0,0,NaN,"['Mark L. Lester'] Betrayal ['Action', 'Drama'..."
45449,0,['Yakov Protazanov'],NaN,227506,Satan Triumphant,['Yermoliev'],['Russia'],en,NaN,"In a small town live two brothers, one a minis...",...,1917,87.0,Released,NaN,0.0,0.0,0,0,NaN,['Yakov Protazanov'] Satan Triumphant nan ['Ye...


In [150]:
#Convert the text from the new column to a matrix of words count
cm = CountVectorizer().fit_transform(data['combine_features'])

In [151]:
#Get the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#print the scores